In [6]:
!pip install googlemaps
!pip install openai
!pip install PolyLine

In [7]:
import pandas as pd
import googlemaps
from datetime import datetime
import folium
import polyline

In [12]:
with open('/content/key.txt', 'r') as file:
    api_key = file.read().strip()

gmaps = googlemaps.Client(key=api_key)

def get_lat_lng(address):
    geocode_result = gmaps.geocode(address)
    location = geocode_result[0]['geometry']['location']
    return location['lat'], location['lng']

homeaddr = 'Mile Square, Indianapolis, IN'

homelatAddr = get_lat_lng(homeaddr)
radius = 10000

ftruck_result= gmaps.places_nearby(
    location=homelatAddr,
    radius=radius,
    keyword='food truck',
    type='food'
)

ftruck_df = pd.DataFrame(ftruck_result['results'])
ftruck_df = ftruck_df[ftruck_df['business_status'] == 'OPERATIONAL']
ftruck_df = ftruck_df[['name', 'place_id','rating','vicinity']]

opening_hours_list = []
website_url_list = []

for pid in ftruck_df['place_id']:
    details = gmaps.place(place_id=pid)

    website_url = details['result'].get('website', 'No website available')
    website_url_list.append(website_url)

    hours = details['result'].get('opening_hours', {}).get('weekday_text', ['No opening hours available'])
    opening_hours_list.append(hours)

ftruck_df['Opening Hours'] = opening_hours_list
ftruck_df['Website'] = website_url_list

ftruck_df = ftruck_df[~ftruck_df['Opening Hours'].apply(lambda x: 'No opening hours available' in x)].reset_index(drop='index')
ftruck_df = ftruck_df.drop(columns=['place_id'])
ftruck_df = ftruck_df.rename(columns={'vicinity':'Address'})
ftruck_df['Opening Hours'] = ftruck_df['Opening Hours'].apply(lambda x: [day.replace('\u202f', '').replace('\u2009', '') for day in x])
ftruck_df.to_excel("./All Nearby Food Trucks 10km.xlsx", engine='openpyxl', index=True)

In [13]:
def extract_weekend_hours(opening_hours_list):
    hours_dict = {}

    for day in opening_hours_list:
        # Check for Saturday and Sunday
        if "Saturday" in day:
            hours_dict["Saturday"] = day.split(": ")[1]
        if "Sunday" in day:
            hours_dict["Sunday"] = day.split(": ")[1]

    return hours_dict

def has_weekend_hours(opening_hours_list):
    # Convert the list to a single string for easier search
    opening_hours_str = " ".join(opening_hours_list)
    # Check for presence of Saturday or Sunday in the string
    return "Saturday" in opening_hours_str or "Sunday" in opening_hours_str

ftruck_df_with_weekend = ftruck_df[ftruck_df['Opening Hours'].apply(has_weekend_hours)].reset_index(drop=True)

ftruck_df['Saturday Hours'] = ftruck_df['Opening Hours'].apply(lambda x: extract_weekend_hours(x).get('Saturday', 'Closed'))
ftruck_df['Sunday Hours'] = ftruck_df['Opening Hours'].apply(lambda x: extract_weekend_hours(x).get('Sunday', 'Closed'))

ftruck_df_with_weekend['Saturday Hours'] = ftruck_df_with_weekend['Opening Hours'].apply(lambda x: extract_weekend_hours(x).get('Saturday', 'Closed'))
ftruck_df_with_weekend['Sunday Hours'] = ftruck_df_with_weekend['Opening Hours'].apply(lambda x: extract_weekend_hours(x).get('Sunday', 'Closed'))

In [15]:
def time_to_minutes(time_str):
    #AM to PM
    time_obj = pd.Timestamp(time_str)
    return time_obj.hour * 60 + time_obj.minute

def is_within_time_range(time_range, preferred_time):
    #Is it within our Breakfast / Lunch / Dinner time range?
    start_time, end_time = time_range.split('–')

    start_minutes = time_to_minutes(start_time.strip())
    end_minutes = time_to_minutes(end_time.strip())

    preferred_minutes = time_to_minutes(preferred_time)

    # Check if the end time is next day
    if end_minutes < start_minutes:
        return start_minutes <= preferred_minutes or preferred_minutes <= end_minutes
    else:
        return start_minutes <= preferred_minutes <= end_minutes

# Filtering for each meal time
breakfast_df = ftruck_df[ftruck_df['Saturday Hours'].apply(is_within_time_range, preferred_time='10:30 AM')].sort_values(by='rating', ascending=False)

# Removing the selected breakfast options from consideration for lunch & dinner
ftruck_df = ftruck_df.drop(breakfast_df.index)
lunch_df = ftruck_df[ftruck_df['Saturday Hours'].apply(is_within_time_range, preferred_time='3:30 PM')].sort_values(by='rating', ascending=False)

# Removing the selected lunch options from consideration for dinner
ftruck_df = ftruck_df.drop(lunch_df.index)
dinner_df = ftruck_df[ftruck_df['Saturday Hours'].apply(is_within_time_range, preferred_time='8:30 PM')].sort_values(by='rating', ascending=False)

breakfast_df.to_excel("./Breakfast Food Trucks.xlsx", engine='openpyxl', index=True)  # index=False prevents writing row indices to the Excel file
lunch_df.to_excel("./Lunch Food Trucks.xlsx", engine='openpyxl', index=True)  # index=False prevents writing row indices to the Excel file
dinner_df.to_excel("./Dinner Food Trucks.xlsx", engine='openpyxl', index=True)  # index=False prevents writing row indices to the Excel file

In [16]:
# Getting the second best options for Saturday
breakfast_saturday_df = breakfast_df.iloc[[1]] if len(breakfast_df) > 1 else None
lunch_saturday_df = lunch_df.iloc[[1]] if len(lunch_df) > 1 else None
dinner_saturday_df = dinner_df.iloc[[1]] if len(dinner_df) > 1 else None

saturday_frames = [df for df in [breakfast_saturday_df, lunch_saturday_df, dinner_saturday_df] if df is not None]
saturday_df = pd.concat(saturday_frames, axis=0)

# Getting the first best options for Sunday (Food must get better after each day)
breakfast_sunday_df = breakfast_df.iloc[[0]] if len(breakfast_df) > 1 else None
lunch_sunday_df = lunch_df.iloc[[0]] if len(lunch_df) > 1 else None
dinner_sunday_df = dinner_df.iloc[[0]] if len(dinner_df) > 1 else None

sunday_frames = [df for df in [breakfast_sunday_df, lunch_sunday_df, dinner_sunday_df] if df is not None]
sunday_df = pd.concat(sunday_frames, axis=0)

saturday_df.to_excel("./Saturday Food Truck List.xlsx", engine='openpyxl', index=True)
sunday_df.to_excel("./Sunday Food Truck List.xlsx", engine='openpyxl', index=True)

In [18]:
def determine_transport_mode(distance_val):
    if distance_val < 1:
        return "walking"
    elif 1 <= distance_val <= 3:
        return "bicycling"
    else:
        return "driving"

def get_initial_distance_and_time(origin, destination):
    #Get distance between origin and destination using walking as default mode.
    distance_result = gmaps.distance_matrix(origins=origin, destinations=destination, mode="walking")
    distance_text = distance_result['rows'][0]['elements'][0]['distance']['text']
    distance_val = float(distance_text.split()[0])
    return distance_val, distance_text

def get_travel_time(origin, destination, mode):
    #Get travel time between origin and destination for a specified mode.
    time_result = gmaps.distance_matrix(origins=origin, destinations=destination, mode=mode)
    time_text = time_result['rows'][0]['elements'][0]['duration']['text']
    return time_text

satAddresses = saturday_df['Address'].tolist()
satAddresses.insert(0, homeaddr)
satAddresses.append(homeaddr)
satlatAddr = [get_lat_lng(address) for address in satAddresses]

sunAddresses = sunday_df['Address'].tolist()
sunAddresses.insert(0, homeaddr)
sunAddresses.append(homeaddr)
sunlatAddr = [get_lat_lng(address) for address in sunAddresses]

def write_to_file(latitudes, addresses, filename):
    with open(filename, 'w') as file:
        for i in range(len(latitudes) - 1):
            distance_val, distance_text = get_initial_distance_and_time(latitudes[i], latitudes[i+1])
            mode = determine_transport_mode(distance_val)
            time_text = get_travel_time(latitudes[i], latitudes[i+1], mode)
            output_str = f"From {addresses[i]} to {addresses[i+1]}: {distance_text} taking {time_text} by {mode}\n"
            file.write(output_str)

write_to_file(satlatAddr, satAddresses, './Saturday Journey Preview.txt')
write_to_file(sunlatAddr, sunAddresses, './Sunday Journey Preview.txt')

In [19]:
import folium
import random

def random_color():
    dark_colors = [
        '#1a1a1a',  # Dark Gray
        '#003366',  # Dark Blue
        '#660033',  # Dark Maroon
        '#003300',  # Dark Green
        '#4B0082',  # Indigo
        '#8B4513',  # Saddle Brown
    ]
    return random.choice(dark_colors)

def visualize_journey_with_dynamic_directions(stops_coordinates, stops_labels, names=None):
    mapcenter = get_lat_lng('Mile Square, Indianapolis, IN')
    map = folium.Map(location=[mapcenter[0], mapcenter[1]], zoom_start=13)

    # Loop through each pair of stops to get individual paths and determine the mode
    for i in range(len(stops_coordinates) - 1):
        distance_val, _ = get_initial_distance_and_time(stops_coordinates[i], stops_coordinates[i+1])
        current_mode = determine_transport_mode(distance_val)

        directions_result = gmaps.directions(origin=stops_coordinates[i],
                                             destination=stops_coordinates[i+1],
                                             mode=current_mode)

        path = directions_result[0]['overview_polyline']['points']
        decoded = polyline.decode(path)
        folium.PolyLine(locations=decoded, color=random_color(), weight=2.5, opacity=1).add_to(map)

    # Adding stops to the map
    for i, (coord, label) in enumerate(zip(stops_coordinates, stops_labels)):
        popup_label = names[i] if names and i < len(names) else label

        folium.CircleMarker(
            location=[coord[0], coord[1]],
            radius=7,
            color='blue',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6,
            popup=popup_label,
        ).add_to(map)

        # Adding numbers for the stops (html style)
        folium.Marker(
            [coord[0], coord[1]],
            icon=folium.DivIcon(
                icon_size=(150, 36),
                icon_anchor=(7, 20),
                html=f'<div style="font-size: 16pt; color: red;">{i+1}</div>',
            ),
            popup=popup_label,
        ).add_to(map)

    return map

In [20]:
def visualize_trip_day(addresses, names, filename):
    # Convert addresses to lat-long coordinates
    latlng_coords = [get_lat_lng(address) for address in addresses]

    # Generate map visualization
    mapvis = visualize_journey_with_dynamic_directions(latlng_coords, addresses, names)
    mapvis.save(filename)
    print(f"Map saved to {filename}")

satNames = ["Home"] + saturday_df['name'].tolist() + ["Home"]
sunNames = ["Home"] + sunday_df['name'].tolist() + ["Home"]

# Call the above function for both Saturday and Sunday
visualize_trip_day(satAddresses, satNames, "./Saturday_Trip.html")
visualize_trip_day(sunAddresses, sunNames, "./Sunday_Trip.html")

Map saved to ./Saturday_Trip.html
Map saved to ./Sunday_Trip.html
